# Unique Faces Comparison Methods

This compares two different methods for finding unique latitude, longitude pairs and assigning them identifiers

In [1]:
import xarray as xr
import numpy as np

In [2]:
def nodes_anissa(corner_lat, corner_lon):

    # Calculate unique
    unique = list(set([(y, x) for x, y in zip(corner_lat, corner_lon)]))
    unique.sort()
    unique_lon = [x[1] for x in unique]
    unique_lat = [x[0] for x in unique]

    nodes = np.reshape([unique.index((y, x)) for x, y in zip(corner_lat, corner_lon)], corner_lon_xr.shape)
    # nodes = np.reshape([unique.index((y, x)) for x, y in zip(corner_lat, corner_lon)], (3, 4))
    
    return nodes

In [3]:
def nodes_orhan(corner_lat, corner_lon):

    corner_lon_lat = np.vstack((corner_lon, corner_lat)).T

    _, unq_ind, unq_inv = np.unique(corner_lon_lat, return_index=True, return_inverse=True, axis=0)

    # Now, calculate unique lon and lat values to account for 'Mesh2_node_x' and 'Mesh2_node_y'
    unq_lon_lat = corner_lon_lat[unq_ind,:]
    unq_lon = unq_lon_lat[:,0]   # out_ds['Mesh2_node_x']
    unq_lat = unq_lon_lat[:,1]   # out_ds['Mesh2_node_y']

    num_faces = int(len(unq_inv)/4)
    nodes = np.reshape(unq_inv, (num_faces,4))  # out_ds['Mesh2_face_nodes']
    
    return nodes
    

In [4]:
# Get data
ne8 = './data/outCSne8.nc'
ds_ne8 = xr.open_dataset(ne8, decode_times=False, engine='netcdf4')  # grid_corner_lat/lon

# Pull out what we need
# Create Mesh2_node_x/y variables from grid_corner_lat/lon
# Turn latitude scrip array into 1D instead of 2D
corner_lat_xr = ds_ne8['grid_corner_lat']
corner_lat = corner_lat_xr.values.ravel()

# Repeat above steps with longitude data instead
corner_lon_xr = ds_ne8['grid_corner_lon']
corner_lon = corner_lon_xr.values.ravel()

# corner_lon = xr.DataArray([[2, 4, 6, 4], [2, 8, 6, 4], [0, 4, 6, 8]]).values.ravel()
# corner_lat = xr.DataArray([[0, 1, 2, 0], [0, 3, 4, 2], [0, 1, 2, 3]]).values.ravel()

In [5]:
# Load line and memory profilers
%load_ext line_profiler
%load_ext memory_profiler

In [6]:
# Time comparison
%timeit -n 100 -r 20 nodes_anissa(corner_lat, corner_lon)
%timeit -n 100 -r 20 nodes_orhan(corner_lat, corner_lon)

9.17 ms ± 245 µs per loop (mean ± std. dev. of 20 runs, 100 loops each)
667 µs ± 24.2 µs per loop (mean ± std. dev. of 20 runs, 100 loops each)


In [7]:
%prun nodes_anissa(corner_lat, corner_lon)

         1560 function calls (1559 primitive calls) in 0.010 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1536    0.009    0.000    0.009    0.000 {method 'index' of 'list' objects}
        1    0.001    0.001    0.009    0.009 1139872182.py:9(<listcomp>)
        1    0.000    0.000    0.010    0.010 1139872182.py:1(nodes_anissa)
        1    0.000    0.000    0.000    0.000 1139872182.py:4(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'sort' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asarray}
        1    0.000    0.000    0.000    0.000 1139872182.py:7(<listcomp>)
        1    0.000    0.000    0.010    0.010 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 1139872182.py:6(<listcomp>)
        1    0.000    0.000    0.010    0.010 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:38(_wrapit)

In [8]:
%prun nodes_orhan(corner_lat, corner_lon)

         116 function calls (110 primitive calls) in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 arraysetops.py:320(_unique1d)
        1    0.001    0.001    0.001    0.001 {method 'argsort' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.ascontiguousarray}
        1    0.000    0.000    0.002    0.002 1798482514.py:1(nodes_orhan)
      9/3    0.000    0.000    0.002    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.002    0.002 arraysetops.py:138(unique)
        1    0.000    0.000    0.000    0.000 {method 'cumsum' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        2    0.000    0.000    0.000    0.000 nume